In [1]:
from transformers import AutoTokenizer, CLIPImageProcessor, CLIPVisionModel, AutoConfig
import numpy as np

from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as T
from torch.cuda.amp import autocast, GradScaler
from vicuna_llava import vicuna_llava, dataset_llava

import zipfile
import wget
from accelerate import Accelerator
accelerator = Accelerator()

Log in to huggingface (using llama requires you to request access on huggingface)

In [2]:

from huggingface_hub import login
login()

Instantiate vicunallava

In [3]:
device = torch.device('cuda:0')

# grab vicuna and its tokenizer
model_name = "meta-llama/Llama-3.2-1B"#"lmsys/vicuna-7b-v1.5"
config = AutoConfig.from_pretrained(model_name)

vicunallava = accelerator.prepare(vicuna_llava(config, llmURL=model_name, accelerator=accelerator))


Grab pretrained projector

In [4]:
#linear_llava_proj = torch.load('mm_projector.bin', weights_only=True)
#linear_llava_weights = linear_llava_proj['model.mm_projector.weight']
#linear_llava_biases = linear_llava_proj['model.mm_projector.bias']

#with torch.no_grad():
#    vicunallava.im_embedding.weight.copy_(linear_llava_weights)
#    vicunallava.im_embedding.bias.copy_(linear_llava_biases)

Grab dataset or subset of dataset

In [5]:
# directories pointing to images directory and chat.json, I've downloaded the dataset locally to deal with some of the missing images
chat = 'CC3M/chat.json'
im_dir = "CC3M/images/"

subset_idcs = list(range(100))

cc3m_dataset = accelerator.prepare(Subset(dataset_llava(chat, im_dir), subset_idcs))

Set which parameters to be optimized

In [6]:
# don't compute gradient on any vicunallava layers besides im_embedding
for i in iter(vicunallava.parameters()):
    i.requires_grad = False
for i in iter(vicunallava.model.parameters()):
    i.requires_grad = False
for i in vicunallava.im_embedding.parameters():
    i.requires_grad = True

Test Generating on Some Sample from Dataset

In [ ]:


#testprompt, testimage, _ = cc3m_dataset[26]
#testimage = testimage.to(device)
#transform = T.ToPILImage()

#output = vicunallava.generate(testimage, testprompt, max_new_tokens=20)
#print(f'input prompt: {testprompt}/n')
#transform(testimage).show()

#print(f'response: {output}')

Training (Stage 1)

In [ ]:
from torch.optim import AdamW

optimizer =  accelerator.prepare(AdamW(vicunallava.parameters(), lr=2e-3))
loss_fn = accelerator.prepare(nn.CrossEntropyLoss(reduction='none', ignore_index=-100))
batch_size=10
losses = []

cc3m_dataloader =  accelerator.prepare(DataLoader(cc3m_dataset, batch_size=batch_size))


vicunallava.train()
for i in range(10000):
    batchiter = 0

    for batchprompt,batchimage,batchresp in cc3m_dataloader:

        input = [batchprompt[i]+'###'+batchresp[i] for i in range(batch_size)]

        tokenized_input = vicunallava.tokenize(input)

        encoded_batchim = vicunallava.vision_tower(batchimage)

        separator_token_id = vicunallava.tokenizer.convert_tokens_to_ids("###")
        with autocast():
            # call model on encoded/tokenized inputs
            outs = vicunallava(batchimage, input, batch_size=batch_size)
            # shift outputs for causal loss calculation
            shifted_outs = outs[:,:-1,:]
            shifted_labels = tokenized_input['input_ids'][:,1:]
            # mask image out of loss computation
            im_loss_mask = torch.full_like(encoded_batchim[:,:,0], -100)
            labels = shifted_labels.clone()
            labels[(labels == separator_token_id).cumsum(dim=1) == 0] = -100
            # loss mask will ensure loss is only computed over the response tokens
            lossmask = torch.cat((im_loss_mask, labels), dim=1).type(torch.LongTensor).to(device)
            # compute loss, append it to list
            loss = loss_fn(shifted_outs.reshape(-1,vicunallava.model.config.vocab_size), lossmask.reshape(-1)).to(device)
            losses.append(loss)

        # compute loss per sample only over non-masked elements
        loss_per_sample = loss[loss != 0].mean()

        # optimize model
        accelerator.backward(loss_per_sample)
        optimizer.step()
        optimizer.zero_grad()
        batchiter+=1
        print(f"iteration:{batchiter}, loss:{loss_per_sample}")
    if i%10 == 0:
        torch.save(vicunallava.im_embedding, 'vicunallava_im_embedding_stage1.pt')


            # takes ~10min for me to locally train through 5 batches of size 20

C:\Users\colli\AppData\Local\Temp\ipykernel_22144\835370053.py:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


iteration:1, loss:11.599148750305176
iteration:2, loss:11.623420715332031
iteration:3, loss:11.605378150939941
iteration:4, loss:11.62193489074707
iteration:5, loss:11.617191314697266
iteration:6, loss:11.574458122253418
iteration:7, loss:11.648216247558594
iteration:8, loss:11.632831573486328
iteration:9, loss:11.567292213439941
iteration:10, loss:11.611964225769043
iteration:1, loss:11.590994834899902
iteration:2, loss:11.624755859375
